##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Day 3 - Function calling with the Gemini API

Welcome back to the Kaggle 5-day Generative AI course!

In this notebook, you will use the Gemini API's automatic function calling to build a chat interface over a local database. This example is a toy and is missing a number of safety and security constraints you would use in a real-world example.

## For help

**Common issues are covered in the [FAQ and troubleshooting guide](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs).**

## Setup

Start by installing and importing the Python SDK.

In [ ]:
#%pip install -q -U 'google-generativeai>=0.8.3'

In [1]:
import google.generativeai as genai

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [2]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

## Create a local database

For this minimal example, you'll create a local SQLite database and add some synthetic data so you have something to query.

Load the `sql` IPython extension so you can interact with the database using magic commands (the `%` instructions) to create a new, empty SQLite database.

In [3]:
%load_ext sql
%sql sqlite:///sample.db

Create the tables and insert some synthetic data. Feel free to tweak this structure and data.

In [4]:
%%sql
-- Create the 'products' table
CREATE TABLE IF NOT EXISTS products (
  	product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	product_name VARCHAR(255) NOT NULL,
  	price DECIMAL(10, 2) NOT NULL
  );

-- Create the 'staff' table
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL
  );

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );

-- Insert data into the 'products' table
INSERT INTO products (product_name, price) VALUES
  	('Laptop', 799.99),
  	('Keyboard', 129.99),
  	('Mouse', 29.99);

-- Insert data into the 'staff' table
INSERT INTO staff (first_name, last_name) VALUES
  	('Alice', 'Smith'),
  	('Bob', 'Johnson'),
  	('Charlie', 'Williams');

-- Insert data into the 'orders' table
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
  	('David Lee', 1, 1),
  	('Emily Chen', 2, 2),
  	('Frank Brown', 1, 3);

 * sqlite:///sample.db
Done.
Done.
Done.
3 rows affected.
3 rows affected.
3 rows affected.


[]

## 定义数据库函数

通过 Gemini API 的 Python SDK 实现函数调用可以通过两种方式:
1. 定义一个[OpenAPI schema](https://ai.google.dev/api/caching#Schema)传递给模型
2. 定义Python函数并让SDK自动检查它们来生成schema

对于后一种方式,重要的是函数要有类型注解和准确的文档字符串来描述函数的功能 - 模型无法了解函数体的内容,所以文档起到了接口的作用。

通过提供三个关键功能 - 列出表格、描述表格和执行查询,LLM(或其他用户)就能获得理解和查询数据库所需的基本工具。

首先创建一个数据库连接,这将在所有函数中共享使用。

In [5]:
import sqlite3

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

第一个函数将列出数据库中所有可用的表。让我们定义这个函数并测试它以确保它正常工作。

In [6]:
def list_tables() -> list[str]:
    """获取数据库中所有表的名称"""
    # 添加打印日志语句,这样你可以看到函数何时被调用
    print(' - DB CALL: list_tables')

    # 创建数据库游标
    cursor = db_conn.cursor()

    # 执行SQL查询获取表名
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    # 获取查询结果并处理
    tables = cursor.fetchall()
    # 将查询结果转换为简单的表名列表并返回
    return [t[0] for t in tables]


# 调用函数测试
list_tables()

 - DB CALL: list_tables


['products', 'sqlite_sequence', 'staff', 'orders']

一旦知道了可用的表,数据库用户下一步需要了解指定表中有哪些列。让我们也定义这个功能,并测试它是否按预期工作。

In [7]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """查询表的结构信息

    Returns:
      返回列信息列表,每个元素是一个元组(列名, 列类型)
    """
    # 添加打印日志,显示函数调用
    print(' - DB CALL: describe_table') 

    # 创建数据库游标
    cursor = db_conn.cursor()

    # 执行PRAGMA命令获取表结构信息
    cursor.execute(f"PRAGMA table_info({table_name});")

    # 获取表结构信息
    schema = cursor.fetchall()
    # schema中每行包含:[列索引, 列名, 列类型, ...]
    # 只返回列名和类型信息
    return [(col[1], col[2]) for col in schema]


In [12]:
tables = list_tables()
print(tables)
# 测试函数 - 查询products表的结构
for table in list_tables():
    describe_table(table)

 - DB CALL: list_tables
['products', 'sqlite_sequence', 'staff', 'orders']
 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: describe_table


Now that the system knows what tables and columns are present, it has enough information to be able to generate and run a `SELECT` query. Now provide that functionality, and test that it works.

In [13]:
def execute_query(sql: str) -> list[list[str]]:
    """执行SELECT语句并返回结果
    
    Args:
        sql: 要执行的SQL查询语句
    Returns:
        查询结果的二维列表 
    """
    # 添加打印日志,显示函数调用
    print(' - DB CALL: execute_query')

    # 创建数据库游标
    cursor = db_conn.cursor()

    # 执行SQL查询
    cursor.execute(sql)
    # 获取并返回所有查询结果
    return cursor.fetchall()


In [14]:
# 测试函数 - 查询products表的所有数据
execute_query("select * from products")

 - DB CALL: execute_query


[(1, 'Laptop', 799.99), (2, 'Keyboard', 129.99), (3, 'Mouse', 29.99)]

## 实现 function calls

现在你可以将所有内容整合到 Gemini API 的调用中。

Function calling 的工作原理是在聊天会话中添加特定的消息。当 function schemas 被定义并提供给模型,且对话开始后,模型可能会返回一个 `function_call` 而不是文本响应。当这种情况发生时,客户端必须以 `function_response` 作为响应,表明调用的结果,然后对话可以正常继续。

这种 function calling 交互通常是手动进行的,允许你(客户端)验证并发起调用。但是 Python SDK 也支持 **automatic function calling**,提供的函数将被自动调用。这是一个强大的功能,只有在安全的情况下才应该使用,比如当函数没有 [side-effects](https://en.wikipedia.org/wiki/Side_effect_(computer_science)) 时。

下面是表示 function calling 对话流程的状态图。在 automatic function calling 中,底部一行由 Python SDK 自动执行。在 manual function calling 中,你需要编写代码来单独运行每个步骤。

![function calling state diagram](https://codelabs.developers.google.com/static/codelabs/gemini-function-calling/img/gemini-function-calling-overview_1440.png)

In [15]:
# 引用之前定义的Python函数
db_tools = [list_tables, describe_table, execute_query]

# 定义系统指令,说明chatbot的功能和行为
instruction = """You are a helpful chatbot that can interact with an SQL database for a computer store. You will take the users questions and turn them into SQL queries using the tools available. Once you have the information you need, you will answer the user's question using the data returned. Use list_tables to see what tables are present, describe_table to understand the schema, and execute_query to issue an SQL SELECT query."""

# 创建Gemini模型实例,配置模型名称、工具函数和系统指令
model = genai.GenerativeModel(
    "models/gemini-1.5-flash-latest", 
    tools=db_tools, 
    system_instruction=instruction
)

# 导入重试策略相关模块
from google.api_core import retry

# 定义重试策略
# 模型可能会自动进行多次连续调用以完成复杂查询
# 当遇到配额限制时确保客户端进行重试
retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# 启动聊天会话,启用自动函数调用功能
chat = model.start_chat(enable_automatic_function_calling=True)

Now you can engage in a chat conversation where you can ask about the contents of the database.

In [16]:
resp = chat.send_message("What is the cheapest product?", request_options=retry_policy)
print(resp.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: execute_query
The cheapest product is Mouse. 



如果你重复使用同一个 [`ChatSession`](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/ChatSession.md) 对象,对话将保持状态继续进行。如果你想重新开始,可以再次调用 [`start_chat`](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md#start_chat),或者在聊天对象上调用 [`rewind`](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/ChatSession.md#rewind) 来回退一个回合。

在这里继续聊天,通过提出后续问题。请注意数据库信息会被保留,并且可以推断出特定产品的上下文。

In [17]:
resp = chat.send_message("and how much is it?", request_options=retry_policy)
print(resp.text)

 - DB CALL: execute_query
It is $29.99. 



探索聊天会话并提出你自己的问题。如果你想尝试提出更复杂的问题,可以试试使用 `gemini-1.5-pro` 模型。在免费版本中,由于它的速率限制较低,调用可能需要更长时间,但你会注意到表达能力有所提升。

In [18]:
model = genai.GenerativeModel(
    "models/gemini-1.5-pro-latest",
    tools=db_tools,
    system_instruction=instruction
)

chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('Which salesperson sold the cheapest product?', request_options=retry_policy)
print(response.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: execute_query
Alice Smith sold the cheapest product.


### 检查会话

要查看模型发出的调用以及客户端返回的响应,你可以检查 `chat.history`。这个辅助函数会打印出每个回合以及传递或返回的相关字段。

In [19]:
# 导入文本包装模块,用于格式化输出
import textwrap

def print_chat_turns(chat):
    """打印出聊天历史中的每个回合,包括函数调用和响应。"""
    
    # 遍历聊天历史中的每个事件
    for event in chat.history:
        # 打印角色名称(首字母大写)
        print(f"{event.role.capitalize()}:")

        # 遍历事件中的每个部分
        for part in event.parts:
            # 如果是文本内容,打印文本
            if txt := part.text:
                print(f'  "{txt}"')
            # 如果是函数调用,打印函数名和参数 
            elif fn := part.function_call:
                # 将函数参数格式化为 key=value 的形式
                args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
                print(f"  Function call: {fn.name}({args})")
            # 如果是函数响应,打印响应内容
            elif resp := part.function_response:
                print("  Function response:")
                # 使用textwrap缩进打印响应
                print(textwrap.indent(str(resp), "    "))

        # 打印空行分隔不同回合
        print()


In [20]:
# 调用函数打印聊天历史
print_chat_turns(chat)

User:
  "Which salesperson sold the cheapest product?"

Model:
  Function call: list_tables()

User:
  Function response:
    name: "list_tables"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              string_value: "products"
            }
            values {
              string_value: "sqlite_sequence"
            }
            values {
              string_value: "staff"
            }
            values {
              string_value: "orders"
            }
          }
        }
      }
    }


Model:
  Function call: describe_table(table_name=products)

User:
  Function response:
    name: "describe_table"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              list_value {
                values {
                  string_value: "product_id"
                }
                values {
                  string_value: "INTEGER"
                }


通过这个输出，你可以清晰地看到每一轮对话的内容和细节。一个重要的特点是，模型的记忆仅限于当前的 `ChatSession` 会话中，这意味着每次新的会话都是一个全新的开始。这种设计带来了一个很大的优势：当你需要修改数据库中的内容或结构时，模型会自动适应这些变化，始终基于最新的数据来响应，而不需要你修改任何代码。建议你实际操作一下，亲身体验这个便捷的特性！

## 延伸阅读

要了解更多关于 Gemini API 在函数调用方面的功能，请查看 [Function calling cookbook](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb) (参考 `Manual function calling` 部分来理解函数调用的手动操作方式)，以及 [Function calling config](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling_config.ipynb)，它可以让你对函数调用的触发方式进行更精细的控制。